In [1]:
import os
import re
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils
mpl_stylesheet.banskt_presentation(splinecolor = 'black', dpi = 120, colors = 'kelly')

In [2]:
data_dir = "/gpfs/commons/home/sbanerjee/work/npd/PanUKB/data"
result_dir = "/gpfs/commons/home/sbanerjee/work/npd/PanUKB/results/nnsparsh/noRx"
# h2_cut = 0.1
# pval_cut = 5e-8

zscore_df = pd.read_pickle(os.path.join(data_dir, f"modselect/zscore_noRx.pkl"))
# trait_df  = pd.read_pickle(os.path.join(data_dir, f"modselect/traits_noRx.pkl"))
trait_df  = pd.read_pickle(os.path.join(data_dir, f"modselect/traits_all_with_desc.pkl"))

variant_filename = f"{data_dir}/allvar.pruned.closesttss.hugo"
variant_df = pd.read_csv(variant_filename, sep = '\t')

In [3]:
def do_standardize(Z, axis = 0, center = True, scale = True):
    '''
    Standardize (divide by standard deviation)
    and/or center (subtract mean) of a given numpy array Z

    axis: the direction along which the std / mean is aggregated.
        In other words, this axis is collapsed. For example,
        axis = 0, means the rows will aggregated (collapsed).
        In the output, the mean will be zero and std will be 1
        along the remaining axes.
        For a 2D array (matrix), use axis = 0 for column standardization
        (with mean = 0 and std = 1 along the columns, axis = 1).
        Simularly, use axis = 1 for row standardization
        (with mean = 0 and std = 1 along the rows, axis = 0).

    center: whether or not to subtract mean.

    scale: whether or not to divide by std.
    '''
    if scale:
        Znew = Z / np.std(Z, axis = axis, keepdims = True)
    else:
        Znew = Z.copy()

    if center:
        Znew = Znew - np.mean(Znew, axis = axis, keepdims = True)

    return Znew

def get_principal_components(X):
    X_cent = do_standardize(X, scale = False)
    X_cent /= np.sqrt(np.prod(X_cent.shape))
    U, S, Vt = np.linalg.svd(X_cent, full_matrices = False)
    loadings = U @ np.diag(S)
    factors  = Vt.T
    return U, S, loadings, factors

def compute_cos(xmat):
    xmat2 = xmat ** 2
    return xmat2 / np.sum(xmat2, axis = 1, keepdims = True)

def compute_contribution(xmat):
    xmat2 = xmat ** 2
    return xmat2 / np.sum(xmat2, axis = 0, keepdims = True)

In [4]:
res_filename = os.path.join(result_dir, "nnm_model_r65536_iter1000.pkl")
with (open(res_filename, "rb")) as fh:
    nnm_model = pickle.load(fh)
    
res_filename = os.path.join(result_dir, "nnm_sparse_model_r65536_iter1000.pkl")
with (open(res_filename, "rb")) as fh:
    nnm_sparse_model = pickle.load(fh)

res_filename = os.path.join(result_dir, "rpca_model.pkl")
with (open(res_filename, "rb")) as fh:
    rpca_model = pickle.load(fh)

In [5]:
nnm_lowX = nnm_model['X_']
nnm_sparse_lowX = nnm_sparse_model['X_']
rpca_lowX = rpca_model['L_']
X = np.array(zscore_df.values.T)
X_cent = X - np.mean(X, axis = 0, keepdims = True)

In [6]:
U_nnm, S_nnm, loadings_nnm, factors_nnm = get_principal_components(nnm_lowX)
U_nnm_sparse, S_nnm_sparse, loadings_nnm_sparse, factors_nnm_sparse = get_principal_components(nnm_sparse_lowX)
U_rpca, S_rpca, loadings_rpca, factors_rpca = get_principal_components(rpca_lowX)

cos2_pheno_nnm   = compute_cos(loadings_nnm)
cos2_variant_nnm = compute_cos(factors_nnm)
contribution_pheno_nnm   = compute_contribution(loadings_nnm)
contribution_variant_nnm = compute_contribution(factors_nnm)

cos2_pheno_nnm_sparse   = compute_cos(loadings_nnm_sparse)
cos2_variant_nnm_sparse = compute_cos(factors_nnm_sparse)
contribution_pheno_nnm_sparse   = compute_contribution(loadings_nnm_sparse)
contribution_variant_nnm_sparse = compute_contribution(factors_nnm_sparse)

cos2_pheno_rpca   = compute_cos(loadings_rpca)
cos2_variant_rpca = compute_cos(factors_rpca)
contribution_pheno_rpca   = compute_contribution(loadings_rpca)
contribution_variant_rpca = compute_contribution(factors_rpca)

In [7]:
loadings_rpca.shape

(2110, 2110)

In [8]:
factors_rpca.shape

(51368, 2110)

In [12]:
X_cent.shape

(2110, 51368)

In [14]:
nsample_filename = "/gpfs/commons/home/sbanerjee/work/npd/PanUKB/data/phe2483.SampleN.tsv"
nsample_df = pd.read_csv(nsample_filename, sep = '\t')

In [24]:
pheno_zindex = [int(x[1:]) for x in zscore_df.columns]
trait_df_noRx = trait_df.loc[trait_df['zindex'].isin(pheno_zindex)]
nsample_df_noRx = nsample_df.loc[trait_df_noRx.index]
trait_df_noRx.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2110 entries, 0 to 2482
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   zindex                       2110 non-null   int64  
 1   trait_type                   2110 non-null   object 
 2   phenocode                    2110 non-null   object 
 3   pheno_sex                    2110 non-null   object 
 4   coding                       267 non-null    object 
 5   modifier                     394 non-null    object 
 6   description                  2110 non-null   object 
 7   description_more             1408 non-null   object 
 8   coding_description           261 non-null    object 
 9   category                     2072 non-null   object 
 10  BIN_QT                       2110 non-null   object 
 11  n_cases_EUR                  2110 non-null   int64  
 12  n_controls_EUR               1304 non-null   float64
 13  N                      

,N
0,420531
1,420531
2,420531
3,420531
4,420531
...,...
2478,418817
2479,401867
2480,401570
2481,402031


In [51]:
pseudoN = np.sum(cos2_pheno_rpca[:, :400] * nsample_df_noRx.to_numpy(), axis = 0, keepdims = True)

In [52]:
pseudoN.shape

(1, 400)

In [57]:
np.array(factors_rpca[:, :400] * np.sqrt(pseudoN))[:, 0]

array([  5.77979753,   3.66005027,   9.91770097, ...,   1.87431437,
       -16.99218782,  -2.04344847])

In [71]:
def create_sumstat(cos2_pheno, factors):
    pseudoN = np.sum(cos2_pheno * nsample_df_noRx.to_numpy(), axis = 0, keepdims = True)
    pseudoZ = factors * np.sqrt(pseudoN)
    return pseudoZ, pseudoN # p x k, n x 1

sumstat, pseudoN = create_sumstat(cos2_pheno_rpca, factors_rpca)
varinfo = variant_df.loc[zscore_df.index][["SNP", "chr", "pos", "ref", "alt"]].rename(columns = {"chr": "CHR", "pos": "BP", "ref": "A1", "alt": "A2"})

In [76]:
pseudoN.shape

(1, 2110)

In [83]:
out_dir = "/gpfs/commons/home/sbanerjee/npddata/panukb/results/ldsc/rpca/sumstats"
for k in range(10):
    varinfo_k = varinfo.copy()
    varinfo_k["Z"] = sumstat[:, k]
    varinfo_k["N"] = pseudoN[0, k]
    outfilename = os.path.join(out_dir, f"factor_{k}.gz")
    varinfo_k.to_csv(outfilename, sep = "\t", header = True, index = False)

In [ ]:
# Create sumstat
Z = W[[factor]] * sqrt(Fn[[factor]])
N = Fn[[factor]]


create_file <- function(fac, Fn, W, snpinfo){

  sumstats <- snpinfo %>% mutate(Z = W[[fac]] * sqrt(Fn[[fac]]),
                                 N = Fn[[fac]])

  return(sumstats)
}

############# write out file for each line of param file #############

for (fac in 1:K){

  if (isTRUE(pseudoN)){
    dat <- create_file(fac, Fn, l2_mat, snpinfo)
  }

  outname <- ifelse(method == "vb",
                    paste0(method, ".scale", vbscale, ".K", K, ".F", fac, ".pseudoN", pseudoN, ".wt_cosine.gz"),
                    paste0(method, ".scale", svdscale, ".K", K, ".F", fac, ".pseudoN", pseudoN, ".gz"))

  if (sum(dat$Z < 0.001 * max(dat$N)) == nrow(dat)){
    dat %>% write_tsv(paste0(outpath, outname))
    print(paste0("Finished: ", fac))
  }else{
    print("has outlier Z that will be removed from ldsc")
  }
}